In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
    #for filename in filenames:
        #print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
df = pd.read_csv('/kaggle/input/glaucoma-detection/glaucoma.csv')
df.head()

,Filename,ExpCDR,Eye,Set,Glaucoma
0,001.jpg,0.7097,OD,A,0
1,002.jpg,0.6953,OS,A,0
2,003.jpg,0.9629,OS,A,0
3,004.jpg,0.7246,OD,A,0
4,005.jpg,0.6138,OS,A,0


In [3]:
df.describe(include='all')

,Filename,ExpCDR,Eye,Set,Glaucoma
count,650,650.000000,650,650,650.000000
unique,650,NaN,2,2,NaN
top,001.jpg,NaN,OD,A,NaN
freq,1,NaN,327,325,NaN
mean,NaN,0.576308,NaN,NaN,0.258462
std,NaN,0.116395,NaN,NaN,0.438126
min,NaN,0.161000,NaN,NaN,0.000000
25%,NaN,0.496675,NaN,NaN,0.000000
50%,NaN,0.567650,NaN,NaN,0.000000
75%,NaN,0.648475,NaN,NaN,1.000000


In [4]:
df['Eye'].unique()

array(['OD', 'OS'], dtype=object)

In [5]:
df['Set'].unique()

array(['A', 'B'], dtype=object)

In [6]:
df['Glaucoma'].unique()

array([0, 1])

In [7]:
df['eye'] = df['Eye'].map({'OD': 0, 'OS': 1})
df.head()

,Filename,ExpCDR,Eye,Set,Glaucoma,eye
0,001.jpg,0.7097,OD,A,0,0
1,002.jpg,0.6953,OS,A,0,1
2,003.jpg,0.9629,OS,A,0,1
3,004.jpg,0.7246,OD,A,0,0
4,005.jpg,0.6138,OS,A,0,1


In [8]:
df['set'] = df['Set'].map({'A': 0, 'B': 1})
df.head()

,Filename,ExpCDR,Eye,Set,Glaucoma,eye,set
0,001.jpg,0.7097,OD,A,0,0,0
1,002.jpg,0.6953,OS,A,0,1,0
2,003.jpg,0.9629,OS,A,0,1,0
3,004.jpg,0.7246,OD,A,0,0,0
4,005.jpg,0.6138,OS,A,0,1,0


In [9]:
df['image'] = np.nan
df.head()

,Filename,ExpCDR,Eye,Set,Glaucoma,eye,set,image
0,001.jpg,0.7097,OD,A,0,0,0,NaN
1,002.jpg,0.6953,OS,A,0,1,0,NaN
2,003.jpg,0.9629,OS,A,0,1,0,NaN
3,004.jpg,0.7246,OD,A,0,0,0,NaN
4,005.jpg,0.6138,OS,A,0,1,0,NaN


In [10]:
image_dir = '/kaggle/input/glaucoma-detection/Fundus_Train_Val_Data/Fundus_Scanes_Sorted'

In [11]:
train_dir = os.path.join(image_dir, 'Train')
validation_dir = os.path.join(image_dir, 'Validation')

In [12]:
mapping = {
    0: 'Glaucoma_Negative',
    1: 'Glaucoma_Positive'
}

In [13]:
from skimage.io import imread
from skimage.transform import resize
import matplotlib.pyplot as plt
%matplotlib inline

In [14]:
train_image_negative = os.listdir(os.path.join(train_dir, 'Glaucoma_Negative'))
train_image_positive = os.listdir(os.path.join(train_dir, 'Glaucoma_Positive'))
print(len(train_image_negative) + len(train_image_positive))

520


In [15]:
test_image_negative = os.listdir(os.path.join(validation_dir, 'Glaucoma_Negative'))
test_image_positive = os.listdir(os.path.join(validation_dir, 'Glaucoma_Positive'))
print(len(test_image_negative) + len(test_image_positive))

130


In [16]:
df_train = df[df['Filename'].isin(train_image_negative) | df['Filename'].isin(train_image_positive)]
len(df_train)

520

In [17]:
df_test = df[df['Filename'].isin(test_image_negative) | df['Filename'].isin(test_image_positive)]
len(df_test)

130

In [18]:
df_train['Glaucoma'].value_counts()

0    386
1    134
Name: Glaucoma, dtype: int64

In [19]:
IMAGE_SIZE = (224,224)

In [20]:
def load_image(row):
    global count
    folder_name = mapping[row['Glaucoma']]
    folder_path = os.path.join(train_dir, folder_name)
    image_path = os.path.join(folder_path, row['Filename'])
    im = imread(image_path)
    im = resize(im, IMAGE_SIZE)
    return im

In [21]:
df_train['image'] = df_train.apply(load_image, axis=1)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [22]:
def load_image1(row):
    global count
    folder_name = mapping[row['Glaucoma']]
    folder_path = os.path.join(validation_dir, folder_name)
    image_path = os.path.join(folder_path, row['Filename'])
    im = imread(image_path)
    im = resize(im, IMAGE_SIZE)
    return im

In [23]:
df_test['image'] = df_test.apply(load_image1, axis=1)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [24]:
X_image_train = df_train['image']
X_data_train = df_train[['ExpCDR', 'eye', 'set']]
y_train = df_train['Glaucoma']

In [25]:
X_image_test = df_test['image']
X_data_test = df_test[['ExpCDR', 'eye', 'set']]
y_test = df_test['Glaucoma']

In [26]:
X_image_train_stacked = np.stack(X_image_train.values)
X_image_test_stacked = np.stack(X_image_test.values)

In [27]:
from sklearn.utils.class_weight import compute_class_weight

In [28]:
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weights

array([0.67357513, 1.94029851])

In [29]:
class_weight_dict = {
    0: class_weights[0],
    1: class_weights[1]
}
class_weight_dict

{0: 0.6735751295336787, 1: 1.9402985074626866}

In [30]:
import tensorflow as tf
from tensorflow.keras import Sequential, layers, Model
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.resnet import ResNet101
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.applications.efficientnet import EfficientNetB7
from sklearn.metrics import classification_report

In [31]:
resnet = ResNet50(input_shape=IMAGE_SIZE+(3,), weights='imagenet', include_top=False)
vgg = VGG16(input_shape=IMAGE_SIZE+(3,), weights='imagenet', include_top=False)
xception = Xception(input_shape=IMAGE_SIZE+(3,), weights='imagenet', include_top=False)
resnet101 = ResNet101(input_shape=IMAGE_SIZE+(3,), weights='imagenet', include_top=False)
inception = InceptionV3(input_shape=IMAGE_SIZE+(3,), weights='imagenet', include_top=False)
mobilenet = MobileNetV2(input_shape=IMAGE_SIZE+(3,), weights='imagenet', include_top=False)
efficientnet = EfficientNetB7(input_shape=IMAGE_SIZE+(3,), weights='imagenet', include_top=False)

258088960/258076736 [==============================] - 8s 0us/step


In [32]:
model_dict = {
    'resnet50': resnet,
    'vgg16': vgg,
    'xception': xception,
    'resnet101': resnet101,
    'inception': inception,
    'mobilenet': mobilenet,
    'efficientnetB7': efficientnet
}
model_dict

{'resnet50': <keras.engine.functional.Functional at 0x7f1ec17a4e50>,
 'vgg16': <keras.engine.functional.Functional at 0x7f1ec17a9850>,
 'xception': <keras.engine.functional.Functional at 0x7f1eb0553d50>,
 'resnet101': <keras.engine.functional.Functional at 0x7f1ec17faa50>,
 'inception': <keras.engine.functional.Functional at 0x7f1e4d8b0390>,
 'mobilenet': <keras.engine.functional.Functional at 0x7f1e4d615090>,
 'efficientnetB7': <keras.engine.functional.Functional at 0x7f1e4c7af7d0>}

In [33]:
for key, model in model_dict.items():
    for layer in model.layers:
        layer.trainable = False

In [34]:
for key, model in model_dict.items():
    x = layers.Flatten()(model.output)
    prediction = layers.Dense(1, activation='sigmoid')(x)
    model = Model(inputs=model.input, outputs=prediction)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    print(key)
    print(model.summary())
    model.fit(X_image_train_stacked, y_train, class_weight=class_weight_dict, epochs=10)
    y_pred = model.predict(X_image_test_stacked)
    y_pred = [round(y[0]) for y in y_pred]
    print(classification_report(y_test, y_pred))

resnet50
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
_____________________________________________________________________________________

/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [35]:
data_input = layers.Input(shape=(3,))

In [36]:
x1 = layers.Dense(1, activation='relu')(data_input)
x1 = Model(inputs=data_input, outputs=x1)

In [37]:
x2 = layers.Flatten()(mobilenet.output)
x2 = Model(inputs=mobilenet.input, outputs=x2)

In [38]:
combined = layers.concatenate([x1.output, x2.output])

In [39]:
x_combined = layers.Dense(150, activation='relu')(combined)
x_combined = layers.Dense(50, activation='relu')(x_combined)
x_combined = layers.Dense(1, activation='sigmoid')(x_combined)

In [40]:
model = Model(inputs=[x1.input, x2.input], outputs=x_combined)

In [41]:
model.summary()

Model: "model_9"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         input_6[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 112, 112, 32) 0           bn_Conv1[0][0]                   
____________________________________________________________________________________________

In [42]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [43]:
model.fit([X_data_train, X_image_train_stacked], y=y_train, class_weight=class_weight_dict, epochs=15)

Epoch 1/15
17/17 [==============================] - 3s 33ms/step - loss: 6.3838 - accuracy: 0.5827
Epoch 2/15
17/17 [==============================] - 1s 33ms/step - loss: 0.7707 - accuracy: 0.7712
Epoch 3/15
17/17 [==============================] - 1s 32ms/step - loss: 0.2394 - accuracy: 0.8923
Epoch 4/15
17/17 [==============================] - 1s 32ms/step - loss: 0.1494 - accuracy: 0.9442
Epoch 5/15
17/17 [==============================] - 1s 33ms/step - loss: 0.0760 - accuracy: 0.9769
Epoch 6/15
17/17 [==============================] - 1s 33ms/step - loss: 0.0507 - accuracy: 0.9923
Epoch 7/15
17/17 [==============================] - 1s 33ms/step - loss: 0.0446 - accuracy: 0.9981
Epoch 8/15
17/17 [==============================] - 1s 33ms/step - loss: 0.0210 - accuracy: 1.0000
Epoch 9/15
17/17 [==============================] - 1s 33ms/step - loss: 0.0149 - accuracy: 1.0000
Epoch 10/15
17/17 [==============================] - 1s 33ms/step - loss: 0.0116 - accuracy: 1.0000
Epoch 11/

In [44]:
model.evaluate([X_data_test, X_image_test_stacked], y_test)

5/5 [==============================] - 1s 27ms/step - loss: 1.2019 - accuracy: 0.7000


[1.2019292116165161, 0.699999988079071]

In [45]:
y_pred = model.predict([X_data_test, X_image_test_stacked])
y_pred = [round(y[0]) for y in y_pred]

In [46]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.82      0.76      0.79        96
           1       0.44      0.53      0.48        34

    accuracy                           0.70       130
   macro avg       0.63      0.64      0.63       130
weighted avg       0.72      0.70      0.71       130



In [47]:
x2_vgg = layers.Flatten()(vgg.output)
x2_vgg = Model(inputs=vgg.input, outputs=x2_vgg)

In [48]:
combined_vgg = layers.concatenate([x1.output, x2_vgg.output])

In [49]:
x_combined_vgg = layers.Dense(150, activation='relu')(combined_vgg)
x_combined_vgg = layers.Dense(50, activation='relu')(x_combined_vgg)
x_combined_vgg = layers.Dense(1, activation='sigmoid')(x_combined_vgg)

In [50]:
model_vgg = Model(inputs=[x1.input, x2_vgg.input], outputs=x_combined_vgg)

In [51]:
model_vgg.summary()

Model: "model_11"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 224, 224, 64) 1792        input_2[0][0]                    
__________________________________________________________________________________________________
block1_conv2 (Conv2D)           (None, 224, 224, 64) 36928       block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_pool (MaxPooling2D)      (None, 112, 112, 64) 0           block1_conv2[0][0]               
___________________________________________________________________________________________

In [52]:
model_vgg.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [53]:
model_vgg.fit([X_data_train, X_image_train_stacked], y_train, class_weight=class_weight_dict, epochs=50)

Epoch 1/50
17/17 [==============================] - 2s 66ms/step - loss: 0.7325 - accuracy: 0.5038
Epoch 2/50
17/17 [==============================] - 1s 66ms/step - loss: 0.5709 - accuracy: 0.7154
Epoch 3/50
17/17 [==============================] - 1s 67ms/step - loss: 0.5684 - accuracy: 0.7327
Epoch 4/50
17/17 [==============================] - 1s 65ms/step - loss: 0.5249 - accuracy: 0.7481
Epoch 5/50
17/17 [==============================] - 1s 66ms/step - loss: 0.4777 - accuracy: 0.7615
Epoch 6/50
17/17 [==============================] - 1s 68ms/step - loss: 0.4601 - accuracy: 0.7904
Epoch 7/50
17/17 [==============================] - 1s 68ms/step - loss: 0.5333 - accuracy: 0.7423
Epoch 8/50
17/17 [==============================] - 1s 72ms/step - loss: 0.7469 - accuracy: 0.6308
Epoch 9/50
17/17 [==============================] - 1s 66ms/step - loss: 0.4502 - accuracy: 0.7808
Epoch 10/50
17/17 [==============================] - 1s 65ms/step - loss: 0.3985 - accuracy: 0.8308
Epoch 11/

In [54]:
model_vgg.evaluate([X_data_test, X_image_test_stacked], y_test)

5/5 [==============================] - 0s 51ms/step - loss: 1.1862 - accuracy: 0.6615


[1.1862367391586304, 0.6615384817123413]

In [55]:
y_pred_vgg = model_vgg.predict([X_data_test, X_image_test_stacked])
y_pred_vgg = [round(y[0]) for y in y_pred_vgg]

In [56]:
print(classification_report(y_test, y_pred_vgg))

              precision    recall  f1-score   support

           0       0.76      0.79      0.78        96
           1       0.33      0.29      0.31        34

    accuracy                           0.66       130
   macro avg       0.55      0.54      0.54       130
weighted avg       0.65      0.66      0.65       130

